## Fake News Classifier Using LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/train.csv')

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.shape

(20800, 5)

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
###Drop Nan Values
df=df.dropna()


In [9]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [10]:
## Get the Independent Features
#we dropped the label feature as it was dependent

X=df.drop('label',axis=1)

In [11]:
## Get the Dependent features
y=df['label']

In [12]:
X.shape

(18285, 4)

In [13]:
y.shape

(18285,)

In [14]:
import tensorflow as tf

In [15]:
tf.__version__

'2.12.0'

In [16]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [17]:
### Vocabulary size
#number of unique words
voc_size=5000

### Onehot Representation

In [18]:
messages=X.copy()

In [19]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [20]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
messages.reset_index(inplace=True)

In [22]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [23]:
import nltk
import re
from nltk.corpus import stopwords
#in LSTM we use word embeddings

In [24]:
#downloading stopwords with respect to english that we dont require in corpus
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [25]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [26]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [27]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [28]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1435, 823, 399, 1987, 4852, 2160, 226, 684, 2802, 1259],
 [2712, 2768, 1334, 1492, 4059, 4800, 1915],
 [2783, 1877, 4588, 4669],
 [1781, 4184, 3787, 3883, 2200, 4370],
 [4200, 4059, 3713, 1845, 3340, 1586, 4059, 1869, 1215, 2403],
 [3680,
  1202,
  1751,
  1691,
  1867,
  2780,
  402,
  2405,
  1425,
  1391,
  4445,
  2282,
  1281,
  3543,
  1915],
 [3753, 806, 1063, 2600, 3552, 4864, 330, 2502, 1872, 2726, 716],
 [1750, 223, 3076, 672, 169, 1209, 2780, 1236, 1872, 2726, 716],
 [3568, 327, 2973, 2323, 2426, 702, 78, 1649, 2780, 4170],
 [4379, 3604, 1974, 3637, 2603, 1334, 4109, 2549],
 [4447, 3616, 3425, 1217, 2004, 4266, 3698, 3915, 4960, 2165, 94],
 [3883, 1509, 4852, 702, 2780, 169],
 [2715, 272, 2544, 848, 3596, 1679, 1356, 600, 4518],
 [3599, 2006, 2093, 208, 2127, 3977, 3417, 1872, 2726, 716],
 [1146, 3130, 4011, 909, 2816, 1872, 2726, 716],
 [2411, 4986, 3880, 2012, 262, 4547, 2524, 1872, 2640, 206],
 [2410, 3904, 2768],
 [3020, 2249, 3675, 3787, 2780, 1378, 3541, 1915],
 [449

In [29]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [30]:
onehot_repr[1]

[2712, 2768, 1334, 1492, 4059, 4800, 1915]

### Embedding Representation

In [31]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1435  823  399 ...    0    0    0]
 [2712 2768 1334 ...    0    0    0]
 [2783 1877 4588 ...    0    0    0]
 ...
 [3139 4917 2332 ...    0    0    0]
 [ 672  169 4094 ...    0    0    0]
 [2691 1432  174 ...    0    0    0]]


In [32]:
embedded_docs[1]

array([2712, 2768, 1334, 1492, 4059, 4800, 1915,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [33]:
embedded_docs[0]

array([1435,  823,  399, 1987, 4852, 2160,  226,  684, 2802, 1259,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [34]:
## Creating model
embedding_vector_features=70 ##features representation #each line will be converted to a vector of 40 size
model=Sequential()
#line below created embedding layer
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
#whenever the output is binary our activation technique should be sigmoid
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 70)            350000    
                                                                 
 lstm (LSTM)                 (None, 100)               68400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 418,501
Trainable params: 418,501
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
len(embedded_docs),y.shape

(18285, (18285,))

In [36]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [37]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [39]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 21s 92ms/step - loss: 0.2922 - accuracy: 0.8604 - val_loss: 0.2120 - val_accuracy: 0.9140
Epoch 2/10
192/192 [==============================] - 6s 30ms/step - loss: 0.1169 - accuracy: 0.9546 - val_loss: 0.1993 - val_accuracy: 0.9162
Epoch 3/10
192/192 [==============================] - 3s 16ms/step - loss: 0.0678 - accuracy: 0.9757 - val_loss: 0.2391 - val_accuracy: 0.9193
Epoch 4/10
192/192 [==============================] - 2s 12ms/step - loss: 0.0490 - accuracy: 0.9835 - val_loss: 0.2854 - val_accuracy: 0.9054
Epoch 5/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0329 - accuracy: 0.9890 - val_loss: 0.3138 - val_accuracy: 0.9157
Epoch 6/10
192/192 [==============================] - 2s 11ms/step - loss: 0.0232 - accuracy: 0.9926 - val_loss: 0.4532 - val_accuracy: 0.8979
Epoch 7/10
192/192 [==============================] - 3s 14ms/step - loss: 0.0153 - accuracy: 0.9952 - val_loss: 0.4305 - val_accuracy: 0.895

### Adding Dropout

In [47]:
from tensorflow.keras.layers import Dropout
## Creating model
#dropout reduces overfitting
embedding_vector_features=70
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [48]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 2ms/step


In [49]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve #1 means fake new 0 means not
#and here whenever y pred is greater than 0.6 i will consider it fake

In [50]:
from sklearn.metrics import confusion_matrix

In [51]:
##A confusion matrix is a table that is used to define the performance of a classification algorithm. A confusion matrix visualizes and summarizes the performance of a classification algorithm.

confusion_matrix(y_test,y_pred)

array([[3419,    0],
       [2616,    0]])

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5665285832642917

In [46]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72      3419
           1       0.00      0.00      0.00      2616

    accuracy                           0.57      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.57      0.41      6035



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
